In [7]:
##################### Imports for the processing of the data ###########################
########################################################################################


import time
import csv
import os
import sys
import re
import serial
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyqtgraph as pg
import tkinter as tk
from scipy import signal
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
from moviepy.editor import *
from moviepy.video.io.preview import preview
from moviepy.video.io.bindings import mplfig_to_npimage
from moviepy.editor import VideoFileClip
from PyQt6.QtGui import *
from PyQt6.QtCore import QDir, Qt, QUrl, QSize, pyqtSignal, pyqtSlot
from PyQt6.QtMultimedia import QMediaPlayer
from PyQt6.QtMultimediaWidgets import QVideoWidget
from PyQt6.QtWidgets import *
from PyQt6 import QtCore, QtWidgets, QtGui
from pyqtgraph import LegendItem
from IPython.display import clear_output


In [17]:
################################### Labeling Tool ######################################
########################################################################################
##Description: 
# This tool is getting used to asign the labels to the indivual datapoints. This step
# is crucial for the Macchine Learning. The technique used here is one hot encoding.
#
############################ by Sandro Tobias Müller ###################################
#Information to the used documentation: 
# https://www.pythonguis.com/pyqt6-tutorial/

###########################################################################################
################################## Processing #############################################

previous_button_label = None

class VideoPlayer(QWidget):
    ## Done only has to be included into whole main window
    progressUpdated = pyqtSignal(float)
    currentFrameChanged = pyqtSignal(int)
    framePositionChanged = pyqtSignal(float)
    stateChanged = pyqtSignal(int)
    

    def __init__(self, parent=None):
        super(VideoPlayer, self).__init__(parent)

        self.mediaPlayer = QMediaPlayer()
        

        btnSize = QSize(16, 16)
        videoWidget = QVideoWidget()
        videoWidget.setFixedSize(600, 350)
        self.mediaPlayer.setVideoOutput(videoWidget)

        #Open video button 
        openButton = QPushButton("Open Video")   
        openButton.setToolTip("Open Video File")
        openButton.setStatusTip("Open Video File")
        openButton.setFixedHeight(24)
        openButton.setIconSize(btnSize)
        openButton.setFont(QFont("Noto Sans", 8))
        openButton.setIcon(QIcon.fromTheme("document-open", QIcon("D:/_Qt/img/open.png")))
        openButton.clicked.connect(self.abrir)

        #Play button
        self.playButton = QPushButton()
        self.playButton.setEnabled(False)
        self.playButton.setFixedHeight(24)
        self.playButton.setIconSize(btnSize)
        self.playButton.setIcon(self.style().standardIcon(QStyle.StandardPixmap.SP_MediaPlay))
        self.playButton.clicked.connect(self.play)

        #Playback speed button
        self.speedButton = QPushButton('1.0x')
        self.speedButton.setEnabled(True)
        self.speedButton.setFixedHeight(24)
        self.speedButton.setIconSize(btnSize)
        self.speedButton.setFont(QFont("Noto Sans", 8))
        self.speedButton.clicked.connect(self.changePlaybackSpeed)

        #Viedio slider
        self.positionSlider = QSlider(Qt.Orientation.Horizontal)
        self.positionSlider.setRange(0, 0)
        self.positionSlider.sliderMoved.connect(self.setPosition)

        self.statusBar = QStatusBar()
        self.statusBar.setFont(QFont("Noto Sans", 7))
        self.statusBar.setFixedHeight(14)

        #layout control 
        controlLayout = QHBoxLayout()
        controlLayout.setContentsMargins(0, 0, 0, 0)
        controlLayout.addWidget(openButton)
        controlLayout.addWidget(self.playButton)
        controlLayout.addWidget(self.speedButton)
        controlLayout.addWidget(self.positionSlider)

        layout = QVBoxLayout()
        layout.addWidget(videoWidget)
        layout.addLayout(controlLayout)
        layout.addWidget(self.statusBar)

        self.setLayout(layout)

        #help(self.mediaPlayer)
        self.mediaPlayer.setVideoOutput(videoWidget)
        self.mediaPlayer.playbackStateChanged.connect(self.mediaStateChanged)
        self.mediaPlayer.positionChanged.connect(self.positionChanged)
        self.mediaPlayer.durationChanged.connect(self.durationChanged)
        self.mediaPlayer.errorChanged.connect(self.handleError)
        self.statusBar.showMessage("Ready")

        

    def abrir(self):
        fileName, _ = QFileDialog.getOpenFileName(self, "Select Media",
                ".", "Video Files (*.mp4 *.flv *.ts *.mts *.avi)")

        if fileName != '':
            self.mediaPlayer.setSource(QUrl.fromLocalFile(fileName))
            self.playButton.setEnabled(True)
            self.statusBar.showMessage(fileName)
            self.mediaPlayer.pause()

    def play(self):
        if self.mediaPlayer.playbackState() == QMediaPlayer.PlaybackState.PlayingState:
            self.mediaPlayer.pause()
        else:
            self.mediaPlayer.play()

    def mediaStateChanged(self, state):
        if self.mediaPlayer.playbackState() == QMediaPlayer.PlaybackState.PlayingState:
            self.playButton.setIcon(
                    self.style().standardIcon(QStyle.StandardPixmap.SP_MediaPause))
        else:
            self.playButton.setIcon(
                    self.style().standardIcon(QStyle.StandardPixmap.SP_MediaPlay))
        self.stateChanged.emit(state)

    def positionChanged(self, position):
        self.positionSlider.setValue(position)
        self.currentFrameChanged.emit(position)
        self.framePositionChanged.emit(position)
        

    def durationChanged(self, duration):
        self.positionSlider.setRange(0, duration)

    def setPosition(self, position):
        self.mediaPlayer.setPosition(position)

    def handleError(self):
        self.playButton.setEnabled(False)
        self.statusBar.showMessage("Error: " + self.mediaPlayer.errorString())

    def changePlaybackSpeed(self): 
        speed = self.mediaPlayer.playbackRate()
        if speed == 1.0:
            self.mediaPlayer.setPlaybackRate(0.5)
            self.speedButton.setText('0.5x')
        elif speed == 0.5:
            self.mediaPlayer.setPlaybackRate(0.10)
            self.speedButton.setText('0.10x') 
        else:
            self.mediaPlayer.setPlaybackRate(1.0)
            self.speedButton.setText('1.0x')
    
    def updateVLinePosition(self, position):
        if self.mediaPlayer.playbackState() == QMediaPlayer.PlaybackState.PlayingState:
            seconds = position / 1000  # convert position from milliseconds to seconds
            self.framePositionChanged.emit(seconds)
            

class PlotWidget(QWidget):
    csvSelected = QtCore.pyqtSignal(str)
    vlinePositionChanged = QtCore.pyqtSignal(float)

    def __init__(self, parent=None):
        super().__init__(parent)
        self.setFixedSize(600, 350)
        self.plotWidget = pg.PlotWidget(self)
        self.plotWidget.move(0, 0)
        self.plotWidget.setFixedSize(600, 350)
        self.plotWidget.setBackground('w')
        self.plotWidget.setLabel('left', 'Impedance/mΩ')
        self.plotWidget.setLabel('bottom', 'Time/s')
        self.plotWidget.setTitle('Preprocessed signal')

        self.browseButton = QPushButton('Select .csv', self)
        self.browseButton.move(10, 10)
        self.browseButton.setFixedWidth(100)
        self.browseButton.clicked.connect(self.browseFile)
        self.browseButton.raise_()  # set z-order of browseButton to be higher 

        

        #Create Vertical line 
        self.vpltline = pg.InfiniteLine(pos=0, angle=90, movable=False)
        self.vpltline.setZValue(10)

        self.data = None
   

    def browseFile(self):
        filePath, _ = QFileDialog.getOpenFileName(self, 'Open CSV', os.getenv('HOME'),
                                                   'CSV(*.csv)')
        if filePath != '':
            self.plotData(filePath)
            self.browseButton.hide()
            self.csvSelected.emit(filePath)  # emit signal with file path

    def plotData(self, filePath):
        data = pd.read_csv(filePath)
        self.data = data
        self.plotWidget.clear()
        self.plotWidget.plot(data['# Time/s'], data['Impedance/mOhm'], pen='g')
        self.plotWidget.addItem(self.vpltline)
    
    @QtCore.pyqtSlot(float)
    def updateVlinePosition(self, milliseconds):
        seconds = float((milliseconds / 1000)) 
        if self.data is not None:
            time_col = self.data['# Time/s']
            idx = np.abs(time_col - seconds).argmin()
            x_value = time_col[idx]
            self.vpltline.setValue(x_value)

class LabelWidget(QWidget):
    def __init__(self, player, parent=None):
        super().__init__(parent)

        self.player = player
        self.current_button = None
        self.setFixedSize(600, 500)
        self.player.stateChanged.connect(self.handle_media_status)
        self.label = QLabel() 
        
        self.vertical_lines = []  
        
        
        self.previous_timestamp = None
        self.previous_button_label = None  

        # Create grid layout with six rows and one column
        layout = QGridLayout()
        layout.setSpacing(10)
        layout.setContentsMargins(10, 10, 10, 10)
        layout.setRowStretch(0, 2)
        layout.setRowStretch(1, 1)
        layout.setRowStretch(2, 1)
        layout.setRowStretch(3, 1)
        layout.setRowStretch(4, 1)
        layout.setRowStretch(5, 1)
        

        self.y_Labels = ['L_nothing','L_thumb','L_index','L_middle','L_ring','L_pinki']
        self.button_label_to_index = {label: index for index,
                                       label in enumerate(self.y_Labels)}  
        self.plotWidgets = []
        self.data = None 

        for i in range(6):
            plotWidget = pg.PlotWidget()
            plotWidget.setBackground('w')
            plotWidget.showGrid(x=True, y=True)
            plotWidget.setTitle(self.y_Labels[i])
            layout.addWidget(plotWidget, i, 0)
            self.plotWidgets.append(plotWidget)
        
        self.setLayout(layout)

    def handle_media_status(self, status):
        if self.player.mediaPlayer.mediaStatus() == QMediaPlayer.MediaStatus.EndOfMedia:
            self.video_finished()

    def video_finished(self):
        print("Video finished!")

    def add_vertical_line(self, time, button_label):
        if self.previous_button_label == button_label:
            return
        plot_index = self.button_label_to_index[button_label]
        plotWidget = self.plotWidgets[plot_index]
        vertical_line = pg.InfiniteLine(time, angle=90, pen=pg.mkPen('r', width=1))
        plotWidget.addItem(vertical_line)
        self.vertical_lines.append(vertical_line)

        if self.previous_timestamp is not None and self.previous_button_label is not None:
            if self.previous_button_label != button_label:  
                previous_plot_index = self.button_label_to_index[self.previous_button_label]
                self.draw_filled_square(self.previous_timestamp, time, previous_plot_index)

    def draw_filled_square(self, start_time, end_time, plot_index):
        if self.current_button is None:
            return
        plotWidget = self.plotWidgets[plot_index]
        start_time = max(plotWidget.getAxis('bottom').range[0], start_time)
        end_time = min(plotWidget.getAxis('bottom').range[1], end_time)

        print(f"start_time: {start_time}, end_time: {end_time}")

        square = QGraphicsRectItem(start_time, 0, end_time - start_time, 1)
        square.setBrush(QColor('green'))
        square.setPen(QPen(Qt.PenStyle.NoPen))
        plotWidget.addItem(square)

    def loadData(self, filePath):
        self.data = pd.read_csv(filePath)  # Assign the read CSV data to the data instance variable
        self.filePath = filePath
        max_time = self.data['# Time/s'].max()
        max_time = max_time + 2 #add two seconds space 
        # fill in the data
        for i in range(6):
            x = self.data['# Time/s']  
            y = self.data[self.y_Labels[i]]
            curve = self.plotWidgets[i].plot(x, y, pen='b', stepMode='left')

            self.plotWidgets[i].setYRange(0,1)
            self.plotWidgets[i].setXRange(-1, max_time)
            curve.setFillLevel(0)
            curve.setFillBrush(pg.mkColor('w').darker(150))
   
    @pyqtSlot(str, float)
    def update_label(self, button_label, current_time):
        self.label.setText(f"Button from Label Widget {button_label} is clicked at {current_time:.3f} seconds.")
        print(f"Button from Label Widget {button_label} is clicked at {current_time:.3f} seconds.")

        if self.previous_button_label != button_label:
            if self.current_button is not None:
                self.draw_filled_square(self.current_button[1], current_time, self.current_button[0])
            
            self.current_button = (self.button_label_to_index[button_label], current_time)
            
            # Update the CSV column with the button label
            self.add_vertical_line(current_time, button_label)
            if button_label in self.data.columns:
                current_index = self.data['# Time/s'].sub(current_time).abs().idxmin()

                # If there's a previous button and it's different from the current button
                if self.previous_button_label is not None and self.previous_button_label != button_label:
                    # Find the index of the previous timestamp
                    previous_index = self.data['# Time/s'].sub(self.previous_timestamp).abs().idxmin()

                    # Write ones to the rows between the previous and current timestamps
                    self.data.loc[previous_index:current_index, self.previous_button_label] = 1

                # Set the current button label and timestamp as the previous button label and timestamp
                self.previous_button_label = button_label
                self.previous_timestamp = current_time

                # Write ones in the column related to the current button from the current timestamp
                self.data.loc[current_index, button_label] = 1

                # Write zeros in other columns for the current timestamp
                for col in self.data.columns:
                    if col != button_label and col != '# Time/s':
                        self.data.loc[current_index, col] = 0
    
    
    def end_function(self):
        print("Window is getting closed")

        ##Trigger last button presses that the ones are getting filled in until the end
        if self.previous_button_label is not None and self.previous_timestamp is not None:
            # Find the index of the last timestamp
            last_index = self.data['# Time/s'].sub(self.previous_timestamp).abs().idxmin()
            print("If was getting triggered")
            # Fill in ones from the last index until the end of the data
            self.data.loc[last_index:, self.previous_button_label] = 1

        ##Save routine from the file 
        filename = os.path.basename(self.filePath)
        # Define the new directory and suffix
        new_directory = r"E:/BachelorThesis_MedTech_Sandro_Mueller/Data/Training_data/Measurements/labeled"
        suffix = "_labeled"
        
        # Combine the new directory, filename, and suffix
        new_filepath = os.path.join(new_directory, filename.replace(".csv", f"{suffix}.csv"))

        # Save the data to the new filepath
        self.data.to_csv(new_filepath, index=False)


class LabelButton(QWidget): 
    buttonClickedSignal = QtCore.pyqtSignal(str, float)

    def __init__(self, video_player):
        super().__init__()

        self.video_player = video_player

        self.previous_button_label = None
        self.previous_timestamp = None

        self.setFixedSize(600, 200)
        self.setWindowTitle('LabelButton')
        self.button_labels = ['L_nothing', 'L_thumb', 'L_index', 'L_middle', 'L_ring', 'L_pinki']
        self.buttons = []
        self.labels = []

        for button in self.buttons:
            button.clicked.connect(self.buttonClicked)
        for i in range(6):
            button = QPushButton(self.button_labels[i], self)
            if i >= 3:
                button.setGeometry(50 + 170 * (i % 3), 90 + 50 * (i // 3), 120, 30)
            else:
                button.setGeometry(50 + 170 * (i % 3), 30 + 50 * (i // 3), 120, 30)
            button.clicked.connect(self.buttonClicked)
            self.buttons.append(button)

            label = QLabel(f'Shortcut {i+1}', self)
            label.setAlignment(Qt.AlignmentFlag.AlignCenter)
            if i >= 3:
                label.setGeometry(50 + 170 * (i % 3), 60 + 50 * (i // 3), 120, 30)
            else:
                label.setGeometry(50 + 170 * (i % 3), 0 + 50 * (i // 3), 120, 30)
            self.labels.append(label)

        for i in range(6):
            shortcut = QShortcut(QKeySequence(str(i+1)), self)
            shortcut.activated.connect(self.buttons[i].click)

        self.buttons[0].click()

    def buttonClicked(self):
        button = self.sender()
        index = self.buttons.index(button)

        current_time = self.video_player.mediaPlayer.position() / 1000
        self.buttonClickedSignal.emit(self.button_labels[index], current_time)

        for i, btn in enumerate(self.buttons):
            if btn == button:
                btn.setStyleSheet('background-color: blue')
            else:
                btn.setStyleSheet('')


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Create instances of your widgets
        self.video_player = VideoPlayer(parent=self)
        self.plot_widget = PlotWidget()
        self.label_widget = LabelWidget(self.video_player)
        self.label_button = LabelButton(self.video_player)

        # Create a main widget and set it as the central widget of the MainWindow
        main_widget = QWidget(self)
        self.setCentralWidget(main_widget)

        self.player = self.video_player.mediaPlayer

        # Create horizontal and vertical layouts to organize the widgets
        main_layout = QHBoxLayout(main_widget)
        top_layout = QHBoxLayout()
        bottom_layout = QHBoxLayout()
        left_layout = QVBoxLayout()
        right_layout = QVBoxLayout()

        # Add the widgets to the appropriate layouts
        left_layout.addWidget(self.video_player)
        right_layout.addWidget(self.label_widget)
        left_layout.addWidget(self.plot_widget)
        right_layout.addWidget(self.label_button)

        top_layout.addLayout(left_layout)
        top_layout.addLayout(right_layout)

        main_layout.addLayout(top_layout)
        main_layout.addLayout(bottom_layout)

        # Connect csvSelected signal to loadData method
        self.plot_widget.csvSelected.connect(self.label_widget.loadData)

        # Connect buttonClickedSignal to update_label method
        self.label_button.buttonClickedSignal.connect(self.label_widget.update_label)

        # Connect vlinePositionChanged signal to updateVLinePosition method
        self.video_player.framePositionChanged.connect(self.plot_widget.updateVlinePosition)

    def closeEvent(self, event):
        self.label_widget.end_function()
        super().closeEvent(event)

if __name__ == '__main__':
    app = 0
    app = QApplication([])
    mainWindow = MainWindow()
    mainWindow.show()
    app.exec()

Button from Label Widget L_nothing is clicked at 0.000 seconds.
Button from Label Widget L_thumb is clicked at 26.566 seconds.
start_time: 0.0, end_time: 26.566
start_time: 0.0, end_time: 26.566
Button from Label Widget L_nothing is clicked at 28.733 seconds.
start_time: 26.566, end_time: 28.733
start_time: 26.566, end_time: 28.733
Button from Label Widget L_nothing is clicked at 29.933 seconds.
Button from Label Widget L_thumb is clicked at 35.333 seconds.
start_time: 28.733, end_time: 35.333
start_time: 28.733, end_time: 35.333
Button from Label Widget L_nothing is clicked at 43.866 seconds.
start_time: 35.333, end_time: 43.866
start_time: 35.333, end_time: 43.866
Button from Label Widget L_nothing is clicked at 50.800 seconds.
Button from Label Widget L_thumb is clicked at 53.033 seconds.
start_time: 43.866, end_time: 53.033
start_time: 43.866, end_time: 53.033
Button from Label Widget L_nothing is clicked at 56.066 seconds.
start_time: 53.033, end_time: 56.066
start_time: 53.033, e

In [ ]:
###################### Labeling Correction and control Tool ############################
########################################################################################
##Description: 
# This tool is used to check the labeled data and be sure that everything is correct
# Inside this tool you have the possiblitie to exchange indivual data labels
#
############################ by Sandro Tobias Müller ###################################
#Information to the used documentation: 
# https://www.pythonguis.com/pyqt6-tutorial/

########################################################################################
############################### Main Window#############################################
class PlotWidget(QWidget):
    def __init__(self, title):
        super().__init__()

        # Create a layout for the plot widget
        layout = QVBoxLayout(self)

        # Create a QLabel for the title
        title_label = QLabel(title)
        title_label.setAlignment(QtCore.Qt.AlignmentFlag.AlignHCenter)
        layout.addWidget(title_label)

        # Create a plot widget using PyQtGraph
        self.plot = pg.PlotWidget()
        self.plot.setBackground('w')
        layout.addWidget(self.plot)
        

class PopupWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Labeling Control Program")
        window_width = 1400
        window_height = 800
        self.resize(window_width, window_height)
        
        # Create a central widget for the layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        
        # Create vertical and horizontal layouts for the left and right sections
        main_layout = QHBoxLayout(central_widget)
        self.left_layout = QVBoxLayout()
        self.right_layout = QHBoxLayout()
        
        # Create a widget for the left section
        left_widget = QWidget()
        left_widget.setLayout(self.left_layout)
        
        # Create a widget for the right section
        right_widget = QWidget()
        right_widget.setLayout(self.right_layout)
        

        # Add the left and right widgets to the main layout
        main_layout.addWidget(left_widget)
        main_layout.addWidget(right_widget)
        
        # Create a QLabel for the file path
        self.file_path_label = QLabel()
        self.file_path_label.setAlignment(QtCore.Qt.AlignmentFlag.AlignHCenter)
        self.file_path_label.setStyleSheet("font-size: 14px; color: blue;")
        self.file_path_label.setFixedHeight(20)

         # Create the "Load CSV" button
        button = QPushButton("Load CSV")
        button.clicked.connect(self.on_button_clicked)

        # Add stretchable spacers to center the button
        self.right_layout.addStretch()
        self.right_layout.addWidget(button)
        self.right_layout.addStretch()
        
        # Initialize the vertical line as None
        self.vertical_line = None
        
        # Create the plot widgets
        titles =["Measurement", "Labeling"]
        self.plot_widgets = []
        for title in titles:
            plot_widget = PlotWidget(title=title)
            self.plot_widgets.append(plot_widget)
            self.left_layout.addWidget(plot_widget) 

        #Calculate the dynamic height and width of the plot Window
        plot_height = int((window_height/2) -50)
        plot_width = int((window_width/2) - 50)
        
        # Set the position and size of the plot widgets
        self.plot_widgets[0].setFixedSize(plot_width, plot_height)
        self.plot_widgets[1].setFixedSize(plot_width, plot_height)
        self.plot_widgets[1].move(20, window_height // 2 + 20)

        # Create the table widget
        self.table_widget = QTableWidget()
        
    def plotting_routine(self,data,headers): 
        # Clear the plots
        self.plot_widgets[0].plot.clear()
        self.plot_widgets[1].plot.clear()
        
        # Plot the data in the first plot
        x = data[headers[0]]
        x = [float(x_value) for x_value in x]
        y_1 = [float(value) for value in data[headers[1]]]
        self.plot_widgets[0].plot.plot(x, y_1, pen='b')  # Plotting in blue

        x_position = 0  # Define start position
        self.vertical_line_1 = pg.InfiniteLine(pos=x_position, angle=90, movable=True,
                                                pen=pg.mkPen(color=(255, 165, 0)))
        self.plot_widgets[0].plot.addItem(self.vertical_line_1)

        # Plot the data in the second plot
        for i in range(2, 8):
            y = [float(value)+2*i for value in data[headers[i]]]
            color = pg.intColor(i - 2)  
            pen = pg.mkPen(color=color, width=2) 
            x_step = np.append(x, x[-1] + 1) #add extra element to ensure StepMode 
            self.plot_widgets[1].plot.plot(x_step, y, stepMode=True, pen=pen)

        #Add the vertical line
        x_position = 0  # Define start position
        self.vertical_line_2 = pg.InfiniteLine(pos=x_position, angle=90, movable=True,
                                                pen=pg.mkPen(color=(255, 165, 0)))
        self.plot_widgets[1].plot.addItem(self.vertical_line_2)  # Add the new vert line

    def data_checker(self, data, headers, groupcheck): 
        headers = headers[2:] #Time and Impedance are not necessary
        # Fill the checker
        for i, header in enumerate(headers):
            self.checker[:, i] = data[header] 

        ################################################
        #  Check the ones groups if they are > 25 ######
        ################################################
        if groupcheck == True:
            for col_idx in range(2, self.checker.shape[1]): #Iterates through columns
                ones_count = 0
                group_start_row = None

                for row_idx in range(self.checker.shape[0]): #Iterates through rows
                    if self.checker[row_idx, col_idx] == 1:
                        if ones_count == 0:
                            group_start_row = row_idx
                        ones_count += 1
                    else:
                        if ones_count < 40 and ones_count > 0:
                            # Mark rows as orange for the group of ones
                            for i in range(group_start_row, row_idx):
                                for col in range(self.table_widget.columnCount()):
                                    item = self.table_widget.item(i, col)
                                    item.setBackground(QtGui.QColor(255, 165, 0))  # Set background color to orange
                        ones_count = 0

                # Handle the case where the last group of ones is at the very end of the column
                if ones_count < 100 and ones_count > 0:
                    for i in range(group_start_row, self.checker.shape[0]):
                        for col in range(self.table_widget.columnCount()):
                            item = self.table_widget.item(i, col)
                            item.setBackground(QtGui.QColor(255, 165, 0))  # Set background color to orange

                print("Finished column", col_idx)

        ################################################
        #  Check if only one 1 is inside every row #####
        ################################################

        for row_idx in range(self.checker.shape[0]):
            row = self.checker[row_idx]

            if np.count_nonzero(row == 1) != 1:
                # Mark the entire row as red
                for col in range(self.table_widget.columnCount()):
                    item = self.table_widget.item(row_idx, col)
                    item.setBackground(QtGui.QColor(255, 0, 0))

        print("Done with marking red")   
        
    def on_item_changed(self, item):
        row = item.row()
        column = item.column()
        value = item.text()
        
        # Update the current data
        header = self.headers[column] 
        self.data[header][row] = value

        # trigger update of the plot 
        self.plotting_routine(self.data, self.headers)

        #Test with checker
        self.data_checker(self.data, self.headers, groupcheck=False)
        
    def on_cell_choose(self, row, column):
        x_range = self.data[self.headers[0]]
        x_set = x_range[row]
        self.vertical_line_1.setPos(x_set)
        self.vertical_line_2.setPos(x_set)
    def resizeEvent(self, event):
        # Update the window width and height values
        self.window_width = event.size().width()
        self.window_height = event.size().height()

        # Call the base class resizeEvent
        super().resizeEvent(event)   

    def on_button_clicked(self):
        self.table_widget.blockSignals(True)
        # Fade away button
        sender = self.sender()
        sender.setVisible(False)
        # Selection of the csv file
        file_dialog = QFileDialog()
        self.file_path, _ = file_dialog.getOpenFileName(self, "Load CSV", "",
                                                         "CSV Files (*.csv)")

        if self.file_path:
            # Process the loaded CSV file
            print("CSV file loaded:", self.file_path)
        else:
            print("No file selected.")
    
        filename = os.path.splitext(os.path.basename(self.file_path))[0]
        self.file_path_label.setText(filename)
        self.left_layout.addWidget(self.file_path_label)
        

        # Loading in the csv file
        with open(self.file_path, 'r') as file:
            reader = csv.DictReader(file)
            
            # Read the header row to get the column names
            self.headers = reader.fieldnames

            headers = self.headers
            # Create a dictionary to store the data for each column
            data = {header: [] for header in headers}
            self.data = data
            
            # Read the data rows and store values in respective variables
            for row in reader:
                for header in headers:
                    if header in data:
                        data[header].append(row[header])

            # trigger inital plot of diagramms 
            self.plotting_routine(self.data,self.headers)
            
            # Create Checker matrix
            self.checker = np.zeros((len(data[headers[0]]), 6))

        
        # Set up for the plot widget 
        self.right_layout.removeWidget(self.table_widget)
        self.table_widget.setMinimumWidth(700)
        self.table_widget.setColumnCount(len(headers))
        self.table_widget.setRowCount(len(data[headers[0]]))
        self.table_widget.setHorizontalHeaderLabels(headers)

        

        for i, header in enumerate(headers):
            for j, value in enumerate(data[header]):
                item = QTableWidgetItem(value)
                self.table_widget.setItem(j, i, item)


        self.right_layout.addWidget(self.table_widget)
        self.table_widget.setSizeAdjustPolicy(QAbstractScrollArea.SizeAdjustPolicy.AdjustToContents)

        self.right_layout.update()

        

        #Add an event manager for cell selection 
        self.table_widget.currentCellChanged.connect(self.on_cell_choose)

        #Add an event manager for value change inside cells 
        self.table_widget.itemChanged.connect(self.on_item_changed)

        # Data checker algorithm is getting triggered for the first time 
        self.data_checker(self.data, self.headers, groupcheck=True)
        
        self.table_widget.blockSignals(False)

    def closeEvent(self, event):
        ## Save the changed csv
        filename = os.path.basename(self.file_path) 
        new_directory = r"E:\Bachelorarbeit_MedTech_Sandro_Mueller\Data\Training_data\Measurements\labeled_controlled" # Define the new directory
        suffix = "_controlled" #Suffix 

        # Combine the new directory, filename, and suffix
        new_filepath = os.path.join(new_directory, filename.replace(".csv", f"{suffix}.csv"))

        # Save the data to the new filepath
        data_frame = pd.DataFrame(self.data) #Convert to dataframe
        data_frame.to_csv(new_filepath, index=False)

        print("File is saved")

        event.accept()

if __name__ == "__main__":
    app = 0
    app = QApplication(sys.argv)
    window = PopupWindow()
    window.show()
    sys.exit(app.exec())

## resetting variables 
%reset -f